# Imports

In [2]:
# imports libs
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Raspando dados

In [6]:
# =========================== Função para scrolling de paginas infinitas =============================== #
# https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7


# Web scrapper for infinite scrolling page 
driver = webdriver.Chrome(executable_path=r"C:\Users\PedroMiyasaki\AppData\Local\SeleniumBasic\chromedriver.exe")
driver.get("https://www.quintoandar.com.br/alugar/imovel/curitiba-pr-brasil/amp")
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

# =========================== Função para scrolling de paginas infinitas =============================== #

In [7]:
# pegando os dados para lopar e montar dataframe depois

# pegar div principal
div_main = driver.find_element_by_xpath("/html/body/main/amp-list/div[2]")

# pegar html da div principal
html_content = div_main.get_attribute('outerHTML')
soup = BeautifulSoup(html_content, 'html.parser')

# pegar a lista de card items de imoveis no site
lista_cards = soup.find_all('div', class_='card item')


# fazer um loop incrimental para pegar os dados
ll = []
c = 0

for i in lista_cards:
    nome = i.select('a span')
    text = lista_cards[c].get_text()
    ll.append(text)
    c += 1


# cópia para testar limpezas dos dados
c = ll

In [9]:
df = {
    'C0': [],
    'C1': [],
    'C2': [],
    'C3': [],
    'C4': [],
    'C5': [],
    'C6': [],
    'C7': [],
    'C8': []
}

# pra cada elemento da lista
for element in c:

    # separe esse elemnt em uma lista própria splitando os "\n"
    element = element.split('\n')

    # e para cada elemento disso
    filtered_elements = []
    for filtered_element in element:
        # coloque na nova lista se ele tiver tamanho suficiente
        if len(filtered_element) > 1:
            filtered_elements.append(filtered_element)

    # agora và nesses elementos com tamanho suficiente
    for item in filtered_elements:

        # faça uma string com eles
        if len(item.strip()) > 1:
            item_str = item.strip().lower()

            # rode vários procedimentos para colocar eles nos campos corretos do dicionário
            if 'anúncio' in item_str:
                df['C0'].append(item)

            elif 'apartamento' in item_str or 'casa' in item_str or 'kitnet' in item_str:
                df['C1'].append(item)

            elif 'exclusivo' in item_str or 'originals' in item_str:
                df['C2'].append(item)

            elif 'curitiba' in item_str:
                df['C4'].append(item)

            elif 'm²' in item_str:
                df['C5'].append(item)

            elif 'dorms' in item_str:
                df['C6'].append(item)

            elif 'aluguel' in item_str:
                df['C7'].append(item)

            elif 'total' in item_str:
                df['C8'].append(item)
            
            # se nenhuma das condições for cumprida, provavelmente se trata de um nome de rua
            else:
                df['C3'].append(item)

    # preencha com 'x' campos vazios da linha que acabou de passar

    # DESCOBRIR CHAVE MAIS COMPRIDA
    len_of_keys = [len(df[key]) for key in df]
    max_key_lenght = max(len_of_keys)

    # PASSAR EM CADA CHAVE VENDO O LEN, SE FOR MENOR QUE  A CHAVE MAIS COMPRIDA, ADD 'X'
    for key in df:
        if len(df[key]) < max_key_lenght:
            df[key].append('vazio') 

df = pd.DataFrame(df)
df.to_csv('raw_data_cwb.csv', index=False)

# Limpeza dos dados

In [131]:
# DATAFRAME TROCADO TROCAD DAG DE LIMPEZA
df = pd.read_csv('raw_data_cwb.csv')

# fix C0 "Novo"
df['C0'] = df['C0'].apply(lambda x: 0 if(x == 'vazio') else 1)

# fix C1 "Tipo"

df['C1'] = df['C1'].apply(lambda x: 'Desconhecido' if (x == 'vazio') else x)

# fix C2 "TipoAnuncio"
df['C2'] = df['C2'].apply(lambda x: x.strip())
df['C2'] = df['C2'].apply(lambda x: 'Nenhum' if (x=='vazio') else x)

# fix C3 "Rua"
df['C3'] = df['C3'].apply(lambda x: 'Desconhecida' if (x=='vazio') else x)

# fix C4 "Bairro" e "Cidade"

df['Bairro'] = df['C4'].apply(lambda x: 'Desconhecido' if (x ==  'vazio') else x.split(',')[0].strip())
df['Cidade'] = df['C4'].apply(lambda x: 'Desconhecido' if (x ==  'vazio') else x.split(',')[1].strip())

# fix C5 "Metragem"
df['C5'] = df['C5'].apply(lambda x: 0 if (x == 'vazio') else x.strip().replace('m²', ''))
df['C5'] = df['C5'].apply(int)

# fix C6 "Comodos"
df['C6'] = df['C6'].apply(lambda x: 0 if (x == 'vazio') else x.replace('dorms', ''))
df['C6'] = df['C6'].apply(int)


# fix c7 "Aluguel"
df['C7'] = df['C7'].apply(lambda x: '0' if (x=='vazio') else x.split('R$')[1])
df['C7'] = df['C7'].apply(lambda x: x.strip())
df['C7'] = df['C7'].apply(lambda x: x.replace('.', ''))
df['C7'] = df['C7'].apply(int)

# fix c8 "Aluguel"
df['C8'] = df['C8'].apply(lambda x: '0' if (x=='vazio') else x.split('R$')[1])
df['C8'] = df['C8'].apply(lambda x: x.strip())
df['C8'] = df['C8'].apply(lambda x: x.replace('.', ''))
df['C8'] = df['C8'].apply(int)

# rename
df.rename(columns={'C0': 'Novo',
                    'C1': 'Tipo',
                    'C2': 'TipoAnuncio',
                    'C3': 'Rua',
                    'C5': 'Metragem',
                    'C6': 'Comodos',
                    'C7': 'Aluguel',
                    'C8': 'Total'}, inplace=True)

# clean df
df = df.loc[df['Aluguel'] > 0]
df = df.loc[df['Total'] > 0]
df = df.loc[df['Comodos'] > 0]
df = df.loc[df['Metragem'] > 0]

df = df.loc[df['Bairro'] != 'Desconhecido']
df = df.loc[df['Cidade'] != 'Desconhecido']

# drop cols
df = df.drop('C4', axis=1)

# Zscore"s gearais, e ZScore médio

In [132]:
df['TicketM.Bairro'] = df[['Total', 'Bairro']].groupby('Bairro').transform('mean')
df['Std.Bairro'] = df[['Total', 'Bairro']].groupby('Bairro').transform('std')


df['TicketM.Comodos'] = df[['Total', 'Comodos']].groupby('Comodos').transform('mean')
df['Std.Comodos'] = df[['Total', 'Comodos']].groupby('Comodos').transform('std')

df['TicketM.Tipo'] = df[['Total', 'Tipo']].groupby('Tipo').transform('mean')
df['Std.Tipo'] = df[['Total', 'Tipo']].groupby('Tipo').transform('std')

df['ZScore.Bairro'] = (df['Total'] - df['TicketM.Bairro']) / df['Std.Bairro']
df['ZScore.Comodos'] = (df['Total'] - df['TicketM.Comodos']) / df['Std.Comodos']
df['ZScore.Tipo'] = (df['Total'] - df['TicketM.Tipo']) / df['Std.Tipo']

df['Zscore'] = (df['ZScore.Bairro'] + df['ZScore.Comodos'] + df['ZScore.Tipo']) / 3

# Distancia da Regressão Linear!

Esse KPI é extremamente interessante, irei intanciar uma regressão linear que irá percorrer todos os imoveis. A previsão dessa regressão linear mostrará qual DEVERIA ser o preço desse imóvel com ele tendo essa variáveis, e é justamente essa diferença doque deveria ser frente ao que é, que dara esse percentual de "desconto" desse imóvel em relação as suas features! 2 Regressões serão feitas, uma delas vai dizer o percentual de desconto em relação a Cidade Toda, e outra em relação somente ao bairro em que ele está

## Modelo Genérico

In [133]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# features
x1 = df[['Novo', 'Metragem', 'Comodos']]
x2 = pd.get_dummies(df[['Tipo', 'TipoAnuncio']])

# scalling and enconding
scaler = StandardScaler()
x1 = pd.concat([x1, x2], axis=1)
x = scaler.fit_transform(x1)

# feature
y = df['Total']

# train model
lr = LinearRegression()
lr.fit(x, y)

# predict
df['PredictOfModel'] = lr.predict(x)

# distance of prediction
df['%PriceOfCityModel'] = df['Total'] / df['PredictOfModel'] * 100

## Modelo por bairro

In [135]:
# fazer dicionários de bairros
lista_bairros = df['Bairro'].drop_duplicates().tolist()
df_bairros = dict((bairro, None) for bairro in lista_bairros)

# fazer dicionário de daframes por bairro
passed = []
for i in range(len(df)):
    if passed.count(df['Bairro'].iloc[i]) < 1:
        dff = df.loc[df['Bairro'] == df['Bairro'].iloc[i]]
        df_bairros[df['Bairro'].iloc[i]] = dff
    else:
        pass

# fazer dicionário de modelos por bairro
modelos = dict((bairro, []) for bairro in lista_bairros)

for key in df_bairros:
    x = df_bairros[key][['Novo', 'Metragem', 'Comodos']]

    # scalling and enconding
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)

    # feature
    y = df_bairros[key]['Total']

    # train model
    lr = LinearRegression()
    lr.fit(x, y)

    # append model
    modelos[key].append(lr)
    modelos[key].append(scaler)

# make prediction
df['PredictOfModelByRegion'] = 0

for rep in range(len(df)):
    # select model and sccaler
    model = modelos[df['Bairro'].iloc[i]][0]
    standard_scaler = modelos[df['Bairro'].iloc[i]][1]

    # make features
    x = df[['Novo', 'Metragem', 'Comodos']].iloc[rep].array.reshape(1, -1)
    x = standard_scaler.transform(x)

    df['PredictOfModelByRegion'].iloc[rep] = model.predict(x)


# distance of prediction
df['%PriceOfRegioModel'] = df['Total'] / df['PredictOfModelByRegion'] * 100

C:\Users\PedroMiyasaki\anaconda3\envs\quinto-andar\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\PedroMiyasaki\anaconda3\envs\quinto-andar\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\PedroMiyasaki\anaconda3\envs\quinto-andar\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\PedroMiyasaki\anaconda3\envs\quinto-andar\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 